In [105]:
import os

while not os.getcwd().endswith("hackathon"):
    os.chdir("..")
    print(f"Now in {os.getcwd()}")

In [106]:
MENU_FOLDER = "competition_data/Menu/"

In [107]:
file_list = os.listdir(MENU_FOLDER)

In [108]:
file_paths = []
for dir_entry in os.scandir(MENU_FOLDER):
    file_paths.append(dir_entry.path)

In [109]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown(file_paths[11])

Processing competition_data/Menu/L Essenza Cosmica.pdf...
[                                        ] (0/6=====[======                                  ] (1/6======[=============                           ] (2/======[====================                    ] (3/6=====[==========================              ] (4/6======[=================================       ] (5/======[========================================] (6/6]


In [110]:
from hackathon.utils.settings.settings_provider import SettingsProvider
import warnings
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

with warnings.catch_warnings(action="ignore"):
    from langchain_ibm import ChatWatsonx
import dotenv

dotenv.load_dotenv();

In [111]:
settings_provider = SettingsProvider()

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

vectorstore = Chroma(
    persist_directory="data/metadata-tests",
    embedding_function=HuggingFaceEmbeddings(
        model_name=settings_provider.get_embeddings_model_name(),
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    ),
)

In [112]:
settings_provider = SettingsProvider()

llm = ChatWatsonx(
    model_id=settings_provider.get_ibm_model_name(),  # type: ignore
    url=settings_provider.get_ibm_endpoint_url(),  # type: ignore
    project_id=settings_provider.get_ibm_project_id(),  # type: ignore
)

In [113]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate


headers_to_split_on = [("#", "header_1"), ("##", "dish_name")]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)
md_header_splits

[Document(metadata={}, page_content='**Ristorante "L\'Essenza Cosmica"**  \nChef Alessandro Stellanova  \nSu Pandora, dove il confine tra la natura rigogliosa e l\'infinito universo si annulla, sorge "L\'Essenza\nCosmica", un ristorante che sfida le leggi conosciute dell\'esperienza culinaria. Qui, lo Chef Alessandro\nStellanova non si limita a cucinare; crea simbiosi tra l\'arte del gusto e l\'energia vivente del pianeta.  \nIl viaggio culinario di Stellanova ebbe origine in un piccolo angolo del suo mondo natale, dove comprese che\nla trasformazione degli ingredienti rispecchiava le strutture essenziali dell\'universo stesso. Questa intuizione\nlo ha portato a perfezionare una straordinaria padronanza della Psionica (Livello V), che utilizza per ideare\nricette che toccano l\'essenza stessa dell\'esistenza. I visitatori descrivono piatti che evocano memorie di vite\nnon vissute e sapori che risuonano in dimensioni intricate.  \nNel suo laboratorio culinario, la Quantistica (Livello 3

In [114]:
from enum import Enum
from pydantic import BaseModel, Field, model_validator
import json


class DishMetadata(BaseModel):
    dish_techniques: list | None = Field(
        description="Techniques involved in making the dish", default_factory=list
    )
    dish_ingredients: list | None = Field(
        description="Ingredients used in the dish", default_factory=list
    )


dish_metadata_keys = list(DishMetadata.model_fields.keys())

In [115]:
llm_with_struct = llm.with_structured_output(DishMetadata)

In [116]:
prompt = """
You are an expert metadata extractor, focused on cuisine. These are the metadata you need to find: {metadata}.
Look for the most relevant metadata values: they usually have strange names and start with capital letters.
Provide only the metadata values, in a format dict like, for example:
{{
    "metadata_name": "value",
    "metadata_name_list": ["value1", "value2"]
}}
nothing else
"""

In [117]:
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        (
            "human",
            "Extract the metadata from the following document {document}",
        ),
    ]
)

In [118]:
metadata_extract_chain = answer_prompt | llm

In [119]:
# from IPython.display import display, Markdown

# display(Markdown(md_header_splits[10].page_content))

In [120]:
t = metadata_extract_chain.invoke(
    {
        "document": md_header_splits[10].page_content,
        "metadata": dish_metadata_keys,
    }
)

In [121]:
DishMetadata.model_validate(json.loads(t.content.replace("\n", "")))

DishMetadata(dish_techniques=['Affumicatura Temporale Risonante', 'Amalgamazione Sintetica Molecolare'], dish_ingredients=['Fusilli del Vento', 'Carne di Drago', 'Carne di Balena Spaziale', 'Uovo di Fenice', 'Pane di Luce', 'Lacrime di Unicorno'])

In [122]:
md_header_splits[10].metadata.update(
    DishMetadata.model_validate(json.loads(t.content.replace("\n", ""))).model_dump()
)

In [123]:
md_header_splits[10]

Document(metadata={'header_1': 'Menu', 'dish_name': 'Nebulosa Celeste di Terrafirma', 'dish_techniques': ['Affumicatura Temporale Risonante', 'Amalgamazione Sintetica Molecolare'], 'dish_ingredients': ['Fusilli del Vento', 'Carne di Drago', 'Carne di Balena Spaziale', 'Uovo di Fenice', 'Pane di Luce', 'Lacrime di Unicorno']}, page_content='Sospeso tra sogno e realtà, il nostro piatto d\'autore, la "Nebulosa Celeste di Terrafirma", è un\'armoniosa\nsinfonia di ingredienti celesti, rafforzato dalla magia delle tecniche culinarie d\'avanguardia che solo lo Chef\nAlessandro Stellanova è in grado di orchestrare.  \nAlla base, un girotondo di Fusilli del Vento abbraccia una generosa porzione di Carne di Drago, infusa con\nuna seducente affumicatura temporale risonante, che concentra secoli di aroma fumé in un istante di\npiacere. Accanto, filetti di Carne di Balena spaziale, lentamente tessuti con Amalgamazione Sintetica\nMolecolare, si poggiano dolcemente, offrendosi in una consistenza che 

In [100]:
from langchain.schema import Document

In [104]:
vectorstore.add_documents(
    documents=[
        Document(
            page_content=md_header_splits[10].page_content,
            metadata=md_header_splits[10].metadata,
        )
    ]
)

ValueError: Expected metadata value to be a str, int, float or bool, got ['Affumicatura Temporale Risonante', 'Amalgamazione Sintetica Molecolare'] which is a list in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [ ]:
for md_header_split in md_header_splits:
    vectorstore.add_texts(
        texts=[md_header_split.page_content],
        metadatas=md_header_split.metadata,
    )